In [2]:
!git clone https://github.com/happy-ditto/HypEmo.git
!git clone https://github.com/fabriceyhc/emoji_translate.git

Cloning into 'HypEmo'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 216 (delta 24), reused 23 (delta 23), pack-reused 182
Receiving objects: 100% (216/216), 3.57 MiB | 6.84 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Cloning into 'emoji_translate'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 94 (delta 49), reused 68 (delta 23), pack-reused 0
Receiving objects: 100% (94/94), 174.79 KiB | 942.00 KiB/s, done.
Resolving deltas: 100% (49/49), done.


In [4]:
import pandas as pd
import random

In [5]:
train_data = pd.read_csv('/content/HypEmo/data/go_emotion/train.csv')

In [6]:
word = pd.read_csv('/content/word2_augmentation.csv')

In [7]:
train_data.head()

,text,aug_text,label
0,why the fuck is bayless isoing,why the fuck is that,2
1,to make her feel threatened,makes her feel threatened.,14
2,dirty southern wankers,"the dirty, dirty southerners of the south.",3
3,omg peyton isn't good enough to help us in the...,december 2015 moon fan,26
4,yes i heard abt the f bombs! that has to be wh...,"thank you for the answer: ""until then, my husb...",15


In [8]:
word.head()

,text,aug_text,label,true_label
0,dirty blond chef,"the dirty, dirty southerners of the south.",3,annoyance
1,demographics? i cannot remember who under 35 r...,i don't know anyone under 35 with cable tv.,6,confusion
2,r/sleeptrain was best used for some sleep trai...,r/sleeptrain may be time for some sleep traini...,5,caring
3,"[name] : same same problem, new simplified com...","same fucking problem, slightly better command ...",3,annoyance
4,"all sounds likely is the former, i could see h...","except for the keys, i can't see how i missed ...",6,confusion


In [10]:
train_data = pd.concat([train_data, word], axis = 1)

In [12]:
train_data.to_csv('/content/train.csv')

In [ ]:
label2idx = {'admiration': 0, 'amusement': 1, 'anger': 2,
      'annoyance': 3, 'approval': 4, 'caring': 5,
      'confusion': 6, 'curiosity': 7, 'desire': 8,
      'disappointment': 9, 'disapproval': 10, 'disgust': 11,
      'embarrassment': 12, 'excitement': 13, 'fear': 14,
      'gratitude': 15, 'grief': 16, 'joy': 17,
      'love': 18, 'nervousness': 19, 'optimism': 20,
      'pride': 21, 'realization': 22, 'relief': 23,
      'remorse': 24, 'sadness': 25, 'surprise': 26}
idx2label = {}
for key, val in label2idx.items():
    idx2label[val] = key
train_data['true_label'] = train_data['label'].replace(idx2label)

train_data_sub = train_data[train_data['true_label'].isin(['annoyance', 'approval', 'caring', 'confusion', 'desire', 'disappointment', 'disapproval', 'excitement', 'nervousness', 'grief', 'pride', 'realization', 'relief'])]

In [ ]:
import os
os.environ["MODEL_DIR"] = '../model'
text = 'The quick brown fox jumps over the lazy dog .'

In [ ]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [ ]:
aug = naw.ContextualWordEmbsAug(
    model_path='roberta-base', action="substitute")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Original:
The quick brown fox jumps over the lazy dog .
Augmented Text:
['The quick thinking fox jumps washing the lazy dog away']


In [ ]:
df['text'][100:110].tolist()

['same but with panic at the disco',
 "got one, it's still bad dude",
 'stuff like this makes me worry a little less about the future generations.',
 "mate you're talking to a remainer who wants to remain... i just want a labour government that's all.",
 "b plug? what the hell. use an a plug or you're never going to become doctor some day.",
 'yea having children will always solve the problem',
 'the contempt from [name] in these threads lately is seriously disappointing.',
 'damn hippies.',
 "i kill them all and then overwrite my only save. i'm bonkers.",
 'being someone who has severe anxiety and in this predicament before, you need to do what’s best for you']

In [ ]:
df['text'][100:110].apply(lambda x: aug.augment(x)[0]).tolist()

['same but slightly cooler than the disco',
 'got good, better still bad *',
 'stuff Big Sandy makes me worry small fraction less about the later generations.',
 "mate you're talking about a lawyer who wants you live i just needed a good offer that's all.",
 'b plug? All the f. use an a plug now so potentially going over become doctor every day.',
 'yea your children will always have no problem',
 'the contempt from Left] in some articles really is seriously astonishing.',
 'damn sure.',
 "i kill them all but randomly overwrite and only kill. i'm overwhelmed.",
 'being someone and has severe anxiety met in this position before, you try to do your fair for kids']

In [ ]:
from tqdm import tqdm
tqdm.pandas()
df = train_data_sub.reset_index(drop = True)
original_text = df['text']
df['text'] = df['text'].progress_apply(lambda x: aug.augment(x)[0])
df = df[~df['text'].isin(original_text)]
# df[df['text'].isin(original_text)]
df.reset_index(drop = True).to_csv("/content/drive/MyDrive/11777-advNLP-group-shared-drive/HW4/word2_augmentation.csv", index = False)

100%|██████████| 8690/8690 [2:50:16<00:00,  1.18s/it]


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("dr")

In [ ]:
df

,text,aug_text,label,true_label
0,dirty blond chef,"the dirty, dirty southerners of the south.",3,annoyance
1,demographics? i cannot remember who under 35 r...,i don't know anyone under 35 with cable tv.,6,confusion
2,r/sleeptrain was best used for some sleep trai...,r/sleeptrain may be time for some sleep traini...,5,caring
3,"[name] : same same problem, new simplified com...","same fucking problem, slightly better command ...",3,annoyance
4,"all sounds likely is the former, i could see h...","except for the keys, i can't see how i missed ...",6,confusion
...,...,...,...,...
8671,[name] was half-blood but it is the smart idea...,"[name] is half blood, but bringing up his fath...",10,disapproval
8672,i just believe that i'm comfortable picking on...,i don't know if i'm comfortable with a religio...,6,confusion
8673,always thought that was awesome but is it real...,"it's always funny, but what do you mean",6,confusion
8674,what are you talking about? Nothing bad that h...,what are you talking about anything bad happen...,3,annoyance


In [ ]:
train_data_sub

,text,aug_text,label,true_label
2,dirty southern wankers,"the dirty, dirty southerners of the south.",3,annoyance
6,demographics? i don’t know anybody under 35 wh...,i don't know anyone under 35 with cable tv.,6,confusion
7,r/sleeptrain might be time for some sleep trai...,r/sleeptrain may be time for some sleep traini...,5,caring
8,"[name] - same fucking problem, slightly better...","same fucking problem, slightly better command ...",3,annoyance
15,"all sounds possible except the key, i can't se...","except for the keys, i can't see how i missed ...",6,confusion
...,...,...,...,...
23477,[name] was half-blood but it wasn't a smart id...,"[name] is half blood, but bringing up his fath...",10,disapproval
23478,i don't know if i'm comfortable with a religio...,i don't know if i'm comfortable with a religio...,6,confusion
23481,always thought that was funny but is it a refe...,"it's always funny, but what do you mean",6,confusion
23482,what are you talking about? anything bad that ...,what are you talking about anything bad happen...,3,annoyance
